<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Modules:
Spark, pyspark, Java.

Bellow it is the old fashion way, risky not Recomended

In [ ]:
""" 
# # Installing Java
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # Dowload Spark
! wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
# # Decompressing archives
! tar xf spark-3.2.1-bin-hadoop2.7.tgz
# # installing findspark
! pip install -q findspark
# # installing pyspark
! pip install pyspark==3.2.1
 """

In [ ]:
""" 
# Importing os library
import os
# # Java envirorment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# # Spark envirorment variable
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
 """

# Better and consistent library that makes the same function as the code above



In [1]:
! pip install pyspark py4j


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


**Importing modules and pandas library that will be used later.**

In [2]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
# import findspark

# findspark.init()

In [3]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

Gooogle Colab Version

In [ ]:
""" 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 
"""

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [4]:
fille = r"C:\Users\video\Desktop\Projetos\API\F-A_root_project\RAW_DATA\Parquet\BrStocksFormated.parquet"

stocks = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [6]:
fille = r"C:\Users\video\Desktop\Projetos\API\F-A_root_project\RAW_DATA\Parquet\Ebit.parquet"

ebit = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [7]:
fille = r"C:\Users\video\Desktop\Projetos\API\F-A_root_project\RAW_DATA\Parquet\Stat.parquet"

stat = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [8]:
fille = r"C:\Users\video\Desktop\Projetos\API\F-A_root_project\RAW_DATA\Parquet\Price.parquet"

price = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [9]:
# Stocks table
stocks.createOrReplaceTempView("General")
# Ebit table
ebit.createOrReplaceTempView("Ebit")
# Stat table
stat.createOrReplaceTempView("Stat")
# Price table
price.createOrReplaceTempView("Price")

# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

In [10]:
# Already called at Calc Table
Gen = spark.sql("""
SELECT * FROM General
WHERE Moeda != 'USD' AND EBITDA != 0 AND ROA != 0 AND ROE != 0;
""").createOrReplaceTempView("Gen")

In [ ]:
# Gen.show()

In [15]:
# Already called at RSCH table
Ebit = spark.sql("""
SELECT * FROM Ebit
where Divida_Liquida != 0;
""").createOrReplaceTempView("Debt")

In [13]:
#Ebit.show()

+---------+----------+--------------+
|   Ticker|      EBIT|Divida_Liquida|
+---------+----------+--------------+
|A2MC34.SA|   1.692E8|      4.4884E9|
|A2MC34.SA| -4.1726E9|      5.4075E9|
|A2MC34.SA|  -8.219E8|      3.8355E9|
| AALR3.SA| 1.27246E8|     5.26468E8|
| AALR3.SA| -1.7508E7|     5.64752E8|
| AALR3.SA|  8.6776E7|     6.50676E8|
|AAPL34.SA| 6.9313E10|     5.9203E10|
|AAPL34.SA| 6.9964E10|      7.442E10|
|AAPL34.SA|1.11852E11|     8.9779E10|
|AAPL34.SA|1.22034E11|     9.6423E10|
|ABTT34.SA|   4.747E9|     1.4279E10|
|ABTT34.SA|   5.514E9|     1.1909E10|
|ABTT34.SA|   8.744E9|       8.251E9|
| AERI3.SA| 1.44614E8|     3.10775E8|
| AERI3.SA| 1.88977E8|     4.58448E8|
| AERI3.SA| 1.83366E8|     5.65811E8|
| AESB3.SA|-2.18234E8|    5.559478E9|
| AGRO3.SA| 2.17969E8|     1.79226E8|
| AGRO3.SA| 1.58777E8|     3.43068E8|
| AGRO3.SA| 7.05361E8|      1.7548E7|
+---------+----------+--------------+
only showing top 20 rows



In [17]:
# Already called at RSCH table
Price = spark.sql("""
SELECT * FROM Price
WHERE valor_de_mercado != 0;
""").createOrReplaceTempView("Prices")

In [ ]:
# Price.show()

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [18]:
Stat = spark.sql("""
SELECT * FROM Stat
WHERE Beta != 0.5;
""")#.createOrReplaceTempView("Stat")

In [19]:
CALCULATE = spark.sql("""
SELECT * FROM Gen
WHERE Margem_Bruta > 0.0 AND Margem_EBITIDA > 0.0 AND 
Margem_Operacional > 0.0 AND Margem_liquida > 0.0 AND 
ROE > 0 AND Crescimento_de_Receita_3T > 0;
""")#.createOrReplaceTempView("Calc")

In [21]:
#CALCULATE.show()

In [22]:
RSCH = spark.sql("""
SELECT p.Ticker, p.valor_de_mercado, p.volume_de_mercado, e.EBIT, e.Divida_Liquida FROM Price as p, Ebit as e
WHERE (p.valor_de_mercado + e.Divida_Liquida)/e.EBIT > 0.0 AND 
p.Ticker == e.Ticker; 
""")#.createOrReplaceTempView("RSCH")

In [ ]:
# DFT = spark.sql("""
# SELECT e.Ticker,
#        Preco_Atual,
#        Fluxo_de_caixa_total,
#        Fluxo_de_caixa_por_acao,
#        EBITDA,
#        Divida_total,
#        Liquidez_imediata,
#        Liquidez_corrente,
#        Receita_total,
#        Divida_Patrimonio,
#        Receita_por_acao,
#        ROA,
#        ROE,
#        Lucro_Bruto,
#        Fluxo_de_Caixa_Livre,
#        Fluxo_de_caixa_operacional,
#        Crescimento_de_Receita_3T,
#        Margem_Bruta,
#        Margem_EBITIDA,
#        Margem_Operacional,
#        Margem_liquida,
#        Moeda,
#        Crescimento_dos_ganhos_3T,
#        EBIT,
#        Divida_Liquida
# FROM General g
# LEFT JOIN Ebit e ON g.Ticker = e.Ticker
# """).createOrReplaceTempView("Filtered")

In [ ]:
# Stat = spark.sql("""
# SELECT f.Ticker,
#        Preco_Atual,
#        Fluxo_de_caixa_total,
#        Fluxo_de_caixa_por_acao,
#        EBITDA,
#        Divida_total,
#        Liquidez_imediata,
#        Liquidez_corrente,
#        Receita_total,
#        Divida_Patrimonio,
#        Receita_por_acao,
#        ROA,
#        ROE,
#        Lucro_Bruto,
#        Fluxo_de_Caixa_Livre,
#        Fluxo_de_caixa_operacional,
#        Crescimento_de_Receita_3T,
#        Margem_Bruta,
#        Margem_EBITIDA,
#        Margem_Operacional,
#        Margem_liquida,
#        Moeda,
#        Crescimento_dos_ganhos_3T,
#        EBIT,
#        Divida_Liquida,
#       Beta,
#       Margem_de_Lucro,
#       Crescimento_de_receitas_4T,
#       Valor_do_ultimo_dividendo
# FROM Filtered f
# LEFT JOIN Stat s ON f.Ticker = s.Ticker;
# """).createOrReplaceTempView("Prstocks")

In [ ]:
# DF = spark.sql("""
# SELECT a.Ticker,
#        Preco_Atual,
#        Fluxo_de_caixa_total,
#        Fluxo_de_caixa_por_acao,
#        EBITDA,
#        Divida_total,
#        Liquidez_imediata,
#        Liquidez_corrente,
#        Receita_total,
#        Divida_Patrimonio,
#        Receita_por_acao,
#        ROA,
#        ROE,
#        Lucro_Bruto,
#        Fluxo_de_Caixa_Livre,
#        Fluxo_de_caixa_operacional,
#        Crescimento_de_Receita_3T,
#        Margem_Bruta,
#        Margem_EBITIDA,
#        Margem_Operacional,
#        Margem_liquida,
#        Moeda,
#        Crescimento_dos_ganhos_3T,
#        EBIT,
#        Divida_Liquida,
#        Beta,
#        Margem_de_Lucro,
#        Crescimento_de_receitas_4T,
#        Valor_do_ultimo_dividendo,
#        valor_de_mercado
# FROM Prstocks a
# LEFT JOIN Price p ON a.Ticker = p.Ticker;
# """).createOrReplaceTempView('Stocks')

ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

In [ ]:
# df_final = CALCULATE.toPandas()

In [ ]:
# df_final.to_excel("/content/gdrive/MyDrive/Data/XLSX/Final/StocksFinal.xlsx",
#                   sheet_name="Stocks")

Pandas setting DataFrame to pandas format:

In [23]:
df_stat = Stat.toPandas()

df_calc = CALCULATE.toPandas()

df_rsc = RSCH.toPandas()

In [24]:
df_stat.to_excel(r"C:\Users\video\Desktop\Projetos\API\F-A_root_project\Final\Stat.xlsx",
                 sheet_name="stat")

df_calc.to_excel(r"C:\Users\video\Desktop\Projetos\API\F-A_root_project\Final\Calcl.xlsx",
                 sheet_name="calc")

df_rsc.to_excel(r"C:\Users\video\Desktop\Projetos\API\F-A_root_project\Final\RSCH.xlsx",
                sheet_name="research")